Arthur Pochic ING3 DS

# Méthode de l'écart interquartile

On import le fichier sonar.all-data.csv et on nomme les colonnes d'observations de F1 à F60 ainsi que la variable cible comme OBJET.

In [53]:
#Aquisition des données
import pandas as pnd
#observations = pnd.read_csv("datas/sonar.all-data.csv")

import pandas as pnd
observations = pnd.read_csv("sonar.all-data.csv", names=["F1","F2","F3","F4","F5","F6","F7","F8","F9",
                                                      "F10","F11","F12","F13","F14","F15","F16","F17","F18","F19",
                                                      "F20","F21","F22","F23","F24","F25","F26","F27","F28","F29",
                                                      "F30","F31","F32","F33","F34","F35","F36","F37","F38","F39",
                                                      "F40","F41","F42","F43","F44","F45","F46","F47","F48","F49",
                                                      "F50","F51","F52","F53","F54","F55","F56","F57","F58","F59",
                                                      "F60","OBJET"])
observations.head(2)
print(observations['OBJET'].value_counts())
print(observations.columns.values)
print(observations.shape)

OBJET
M    111
R     97
Name: count, dtype: int64
['F1' 'F2' 'F3' 'F4' 'F5' 'F6' 'F7' 'F8' 'F9' 'F10' 'F11' 'F12' 'F13'
 'F14' 'F15' 'F16' 'F17' 'F18' 'F19' 'F20' 'F21' 'F22' 'F23' 'F24' 'F25'
 'F26' 'F27' 'F28' 'F29' 'F30' 'F31' 'F32' 'F33' 'F34' 'F35' 'F36' 'F37'
 'F38' 'F39' 'F40' 'F41' 'F42' 'F43' 'F44' 'F45' 'F46' 'F47' 'F48' 'F49'
 'F50' 'F51' 'F52' 'F53' 'F54' 'F55' 'F56' 'F57' 'F58' 'F59' 'F60' 'OBJET']
(208, 61)


In [54]:
observations["OBJET"].head(2)

0    R
1    R
Name: OBJET, dtype: object


Nous allons identifier les valeurs extrêmes présentes dans les lignes du dataset.
Une donnée est considérée comme aberrante lorsque l’écart absolu par rapport à Q1 ou Q3 dépasse 1,5 fois l’écart interquartile (IQR).
Plus concrètement :
- une valeur est dite aberrante faible si elle est inférieure à Q1 - 1.5 * (Q3 - Q1),
- et aberrante élevée si elle est supérieure à Q3 + 1.5 * (Q3 - Q1).

Enfin, nous supprimerons les lignes contenant au moins 7 valeurs aberrantes.


In [55]:
#rendre la colonne Objet numérique
observations['OBJET'] = observations['OBJET'].map({'M': 1, 'R': 0})
print(observations['OBJET'])

0      0
1      0
2      0
3      0
4      0
      ..
203    1
204    1
205    1
206    1
207    1
Name: OBJET, Length: 208, dtype: int64


In [56]:
#On crée la liste des colonnes à traiter
colonnes = [col for col in observations.columns if col != "OBJET"]

#On définit Q1 et Q3 ainsi que IQR
Q1 = observations[colonnes].quantile(0.25)
Q3 = observations[colonnes].quantile(0.75)
IQR = Q3 - Q1



In [57]:
# On parcourt toutes les caractéristiques
import numpy as np

num_lignes = []  # liste qui va contenir les index des données extrêmes

for caracteristique in colonnes:
    print(caracteristique)

    #Pour chaque colonne, on détermine les seuils supérieur et inférieur
    low_bound = Q1[caracteristique] - 1.5 * IQR[caracteristique]
    up_bound = Q3[caracteristique] + 1.5 * IQR[caracteristique]

    #On récupère les indices des lignes pour lesquelles le booléen est True
    indices_extremes = observations[
        (observations[caracteristique] < low_bound) |
        (observations[caracteristique] > up_bound)
    ].index.tolist()

    #On ajoute ces indices à la liste num_lignes
    num_lignes.extend(indices_extremes)

    print(f"liste_donnees_extremes pour la caractéristique {caracteristique} = {indices_extremes}")
    print("------- --------")

# Affichage final des index de toutes les données extrêmes
print("Index de toutes les données extrêmes :", num_lignes)


F1
liste_donnees_extremes pour la caractéristique F1 = [4, 87, 98, 126, 129, 131, 132, 133, 134, 136, 138, 141, 145, 146, 148]
------- --------
F2
liste_donnees_extremes pour la caractéristique F2 = [6, 85, 98, 100, 102, 129, 131, 134, 136, 138, 140, 141, 145]
------- --------
F3
liste_donnees_extremes pour la caractéristique F3 = [6, 85, 98, 100, 102, 111, 129, 138, 141, 142, 148]
------- --------
F4
liste_donnees_extremes pour la caractéristique F4 = [6, 19, 20, 85, 97, 98, 102, 111, 129, 133, 138, 141, 148]
------- --------
F5
liste_donnees_extremes pour la caractéristique F5 = [19, 98, 111, 130, 147]
------- --------
F6
liste_donnees_extremes pour la caractéristique F6 = [1, 19, 20, 90, 91, 130, 138, 143]
------- --------
F7
liste_donnees_extremes pour la caractéristique F7 = [19, 20, 91, 127, 134]
------- --------
F8
liste_donnees_extremes pour la caractéristique F8 = [1, 2, 19, 87, 91, 127, 132, 134, 135, 136, 171]
------- --------
F9
liste_donnees_extremes pour la caractéristiqu

In [58]:
#On ordonne la liste par ordre croissant des index
num_lignes.sort()
print(num_lignes)

#On crée une liste contenant les numéros de lignes à supprimer
num_lignes_a_supprimer_iq=[]

from collections import Counter

#On compte les occurrences par ligne
counts = Counter(num_lignes)

#On sélectionne les lignes avec au moins 7 valeurs aberrantes
num_lignes_a_supprimer_iq = [idx for idx, cnt in counts.items() if cnt > 7]

#On supprime les lignes
cleaned_data_iq = observations.drop(index=num_lignes_a_supprimer_iq).reset_index(drop=True)

[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 6, 6, 16, 16, 16, 16, 17, 17, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 33, 34, 35, 36, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 47, 65, 85, 85, 85, 86, 87, 87, 87, 87, 88, 88, 90, 91, 91, 91, 97, 97, 97, 98, 98, 98, 98, 98, 98, 98, 99, 99, 99, 100, 100, 100, 101, 101, 101, 101, 102, 102, 102, 102, 102, 104, 104, 105, 105, 111, 111, 111, 112, 112, 115, 115, 122, 126, 127, 127, 127, 127, 127, 127, 129, 129, 129, 129, 129, 129, 130, 130, 130, 130, 131, 131, 131, 131, 131, 131, 131, 131, 132, 132, 132, 132, 132, 132, 132, 133, 133, 133, 133, 133, 133, 133, 133, 133, 133, 134, 134, 134, 134, 134, 134, 134, 134, 134, 135, 135, 135, 135, 135, 135, 135, 135, 135, 136, 136, 136, 136, 136, 136, 136, 136, 136, 136, 136, 137, 137, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 140, 140, 140, 140, 140, 140, 140, 140, 140, 140, 140, 141, 141, 141, 141, 141, 141, 141, 142

In [59]:
print(num_lignes_a_supprimer_iq)

[19, 22, 44, 131, 133, 134, 135, 136, 138, 140, 142, 146, 147]


<h1>Travail demandé</h1>
Rédigez un état de l’art sur la détection des valeurs aberrantes (outliers) dans les datasets. Vous devez :

1. Présenter les principales méthodes existantes (statistiques classiques, méthodes robustes, techniques basées sur le machine learning, approches graphiques…).

2. Comparer leurs avantages et limites selon le type de données (univariées, multivariées, volumineuses…).

3. Illustrer chaque méthode par des exemples concrets d’application

# Méthode des écarts types

Nous allons maintenant faire le même travail avec une autre méthode. La méthode des écarts types.

In [60]:
#On introduit le paramètre pour la règle des écarts types. Généralement 3 (≈99.7% des données selon la loi normale).
k = 3

In [ ]:
# On parcourt toutes les caractéristiques
import numpy as np

num_lignes = []  # liste qui va contenir les index des données extrêmes

for caracteristique in colonnes:
    print(caracteristique)

    #Pour chaque colonne, on détermine la moyenne et l'écart type
    mean = observations[caracteristique].mean()
    std = observations[caracteristique].std()

    #On récupère les indices des lignes pour lesquelles le booléen est True
    indices_extremes = observations[
        (observations[caracteristique] < mean-k*std) |
        (observations[caracteristique] > mean+k*std)
    ].index.tolist()

    #On ajoute ces indices à la liste num_lignes
    num_lignes.extend(indices_extremes)

    #print(f"liste_donnees_extremes pour la caractéristique {caracteristique} = {indices_extremes}")
    #print("------- --------")

# Affichage final des index de toutes les données extrêmes
print("Index de toutes les données extrêmes :", num_lignes)

F1
liste_donnees_extremes pour la caractéristique F1 = [98, 129, 131, 134, 136, 146]
------- --------
F2
liste_donnees_extremes pour la caractéristique F2 = [85, 98, 145]
------- --------
F3
liste_donnees_extremes pour la caractéristique F3 = [85, 98, 111, 138]
------- --------
F4
liste_donnees_extremes pour la caractéristique F4 = [98, 111]
------- --------
F5
liste_donnees_extremes pour la caractéristique F5 = [19, 98, 111, 147]
------- --------
F6
liste_donnees_extremes pour la caractéristique F6 = [20, 91]
------- --------
F7
liste_donnees_extremes pour la caractéristique F7 = [91, 134]
------- --------
F8
liste_donnees_extremes pour la caractéristique F8 = [19, 132, 134, 135, 136]
------- --------
F9
liste_donnees_extremes pour la caractéristique F9 = [2, 132, 133, 134, 135, 136]
------- --------
F10
liste_donnees_extremes pour la caractéristique F10 = [2, 133, 137]
------- --------
F11
liste_donnees_extremes pour la caractéristique F11 = [137, 184]
------- --------
F12
liste_donn

In [62]:
#On ordonne la liste par ordre croissant des index
num_lignes.sort()
print(num_lignes)

#On crée une liste contenant les numéros de lignes à supprimer
num_lignes_a_supprimer_et=[]

from collections import Counter

#On compte les occurrences par ligne
counts = Counter(num_lignes)

#On sélectionne les lignes avec au moins 7 valeurs aberrantes
num_lignes_a_supprimer_et= [idx for idx, cnt in counts.items() if cnt > 7]

#On supprime les lignes
cleaned_data_et = observations.drop(index=num_lignes_a_supprimer_et).reset_index(drop=True)

[2, 2, 2, 2, 3, 3, 5, 5, 19, 19, 20, 21, 22, 22, 22, 22, 22, 22, 22, 22, 33, 44, 44, 44, 65, 85, 85, 87, 91, 91, 97, 97, 98, 98, 98, 98, 98, 98, 101, 101, 105, 111, 111, 111, 112, 127, 127, 127, 129, 129, 129, 131, 132, 132, 133, 133, 133, 134, 134, 134, 134, 135, 135, 135, 135, 136, 136, 136, 136, 137, 137, 138, 138, 138, 140, 140, 140, 140, 140, 141, 141, 142, 142, 142, 143, 143, 143, 144, 145, 145, 146, 146, 146, 146, 146, 146, 146, 146, 146, 147, 147, 147, 147, 147, 147, 147, 147, 161, 184, 186, 186, 186]


In [63]:
print(num_lignes_a_supprimer_et)

[22, 146, 147]


# Méthodes des tests statistiques : Test de Grubbs

import numpy as np
from scipy import stats
from collections import Counter

#Définition de la fonction grubbs_test
def grubbs_test(values, alpha=0.05):
    n = len(values)
    mean = np.mean(values)
    std = np.std(values, ddof=1)

    #Statistique G
    abs_diffs = np.abs(values - mean)
    max_idx = np.argmax(abs_diffs)
    G = abs_diffs[max_idx] / std

    #Valeur critique
    t_crit = stats.t.ppf(1 - alpha / (2*n), n-2)
    G_crit = ((n-1)/np.sqrt(n)) * np.sqrt(t_crit**2 / (n-2 + t_crit**2))

    return G, G_crit, max_idx

num_lignes = []

for caracteristique in colonnes:
    indices_outliers_col = []
    data = observations[caracteristique].values
    data_copy = data.copy()
    indices_copy = np.arange(len(data))

    while True:
      G, G_crit, idx = grubbs_test(data_copy, alpha=0.05)
      if G > G_crit:
        #On ajoute l’indice original
        indices_outliers_col.append(int(indices_copy[idx]))
        #On supprime de la copie
        data_copy = np.delete(data_copy, idx)
        indices_copy = np.delete(indices_copy, idx)
      else:
        break

    print(caracteristique)
    print(f"liste_donnees_extremes pour {caracteristique} = {indices_outliers_col}")
    print("------- --------")

    #On ajoute ces indices à la liste num_lignes
    num_lignes.extend(indices_outliers_col)

    # Affichage final des index de toutes les données extrêmes
print("Index de toutes les données extrêmes :", num_lignes)

#On ordonne la liste par ordre croissant des index
num_lignes.sort()
print(num_lignes)

#On crée une liste contenant les numéros de lignes à supprimer
num_lignes_a_supprimer=[]

from collections import Counter

#On compte les occurrences par ligne
counts = Counter(num_lignes)

#On sélectionne les lignes avec au moins 7 valeurs aberrantes
num_lignes_a_supprimer = [idx for idx, cnt in counts.items() if cnt > 7]

#On supprime les lignes
cleaned_data = observations.drop(index=num_lignes_a_supprimer).reset_index(drop=True)

In [64]:
#print(num_lignes_a_supprimer)

# Methodes Robustes : 

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter

# Liste des colonnes numériques à traiter
colonnes = [col for col in observations.columns if col != "OBJET"]

num_lignes = []  # index des données extrêmes

# Parcours des colonnes
for caracteristique in colonnes:
    print(caracteristique)

    # Médiane et MAD
    median = observations[caracteristique].median()
    mad = np.median(np.abs(observations[caracteristique] - median))

    # Z-score robuste
    z_robust = (observations[caracteristique] - median) / (1.4826 * mad)

    # Indices extrêmes selon le seuil (3.5 est une valeur courante)
    indices_extremes = observations[np.abs(z_robust) > 3.5].index.tolist()

    num_lignes.extend(indices_extremes)

    #print(f"liste_donnees_extremes pour {caracteristique} = {indices_extremes}")
    #print("------- --------")

# Index de toutes les données extrêmes
#print("Index de toutes les données extrêmes :", num_lignes)

# Tri
num_lignes.sort()
print(num_lignes)

# Comptage par ligne
counts = Counter(num_lignes)

# Critère : au moins 7 valeurs extrêmes sur la même ligne
num_lignes_a_supprimer_zr = [idx for idx, cnt in counts.items() if cnt > 7]

# Nettoyage des données
cleaned_data_zr = observations.drop(index=num_lignes_a_supprimer_zr).reset_index(drop=True)


F1
liste_donnees_extremes pour F1 = [4, 87, 98, 129, 131, 132, 133, 134, 136, 146]
------- --------
F2
liste_donnees_extremes pour F2 = [85, 98, 102, 129, 131, 136, 138, 140, 141, 145]
------- --------
F3
liste_donnees_extremes pour F3 = [6, 85, 98, 100, 102, 111, 129, 138, 141, 148]
------- --------
F4
liste_donnees_extremes pour F4 = [19, 98, 102, 111, 129, 133, 138, 141, 148]
------- --------
F5
liste_donnees_extremes pour F5 = [19, 98, 111, 130, 147]
------- --------
F6
liste_donnees_extremes pour F6 = [20, 91, 138]
------- --------
F7
liste_donnees_extremes pour F7 = [19, 20, 91, 127, 134]
------- --------
F8
liste_donnees_extremes pour F8 = [1, 2, 19, 91, 132, 134, 135, 136, 171]
------- --------
F9
liste_donnees_extremes pour F9 = [2, 19, 132, 133, 134, 135, 136, 171]
------- --------
F10
liste_donnees_extremes pour F10 = [2, 19, 133, 134, 137, 171, 184]
------- --------
F11
liste_donnees_extremes pour F11 = [137, 184]
------- --------
F12
liste_donnees_extremes pour F12 = []
--

# Comparaison 

In [67]:
print(num_lignes_a_supprimer_zr)
print(num_lignes_a_supprimer_et)
print(num_lignes_a_supprimer_iq)

[22, 140, 146, 147]
[22, 146, 147]
[19, 22, 44, 131, 133, 134, 135, 136, 138, 140, 142, 146, 147]
